This is the notebook for Tutorial 9 of TS2CG. In this tutorial, we will use the base features of the point class in python, which modifies the point folder created by PLM. After each modification, we can update the point folder and run PCG to obstain a structure.

We will read in a membrane, explore the different methods to access different parts of the folder, and modify some inclusions, exclusions and domains.

In [1]:
#We do some imports to help us with our different steps below:
import numpy as np
import matplotlib.pyplot as plt

#And we load TS2CG to get access to the framework
import TS2CG

For this first part of the tutorial, we will start with the point folder created in Tutorial 4. The folder contains a round bilayer vesicle and three protein inclusions of two different types.

In [2]:
point=TS2CG.core.point.Point("./point_tut4_start")

We are given a warning here. Luckily, this is nothing to worry about, as it just informs us that one of our input files is empty. Something we can easily check and confirm, if we want it that way. In our case point_tutorial4/ExcData.dat does not contain any exclusions, which is acceptable. It does, however, contain Inclusions. In this next step, we will add some more.

In [3]:
#First, let's look at the inclusions we got.
point.inclusions.get_all()
# We see a dictionary of all the inclusions, their id, their type and their location on the point grid as the point id.

[{'id': 0,
  'type_id': 1,
  'point_id': 5,
  'orientation': array([ 0.499, -0.864,  0.059])},
 {'id': 1,
  'type_id': 1,
  'point_id': 22,
  'orientation': array([ 0.481, -0.765, -0.429])},
 {'id': 2,
  'type_id': 2,
  'point_id': 30,
  'orientation': array([-0.177, -0.945,  0.275])}]

In [4]:
#Now we add another protein of type 2.
point.inclusions.add_protein(type_id=2,point_id=17)
# We can also set the orientation, but now it will default to [1,0,0].
# Let's look at our inclusions list, but restrain ourselves to proteins of type 2.
point.inclusions.get_by_type(2)

[{'id': 2,
  'type_id': 2,
  'point_id': 30,
  'orientation': array([-0.177, -0.945,  0.275])},
 {'id': 3, 'type_id': 2, 'point_id': 17, 'orientation': array([1., 0., 0.])}]

In [5]:
#Now we overwrite the point folder with our altered version.
point.save()
#Let's run PCG and look at the new inclusion.
!TS2CG PCG -str input.str -Bondlength 0.2 -LLIB Martini3.LIB -defout system -dts point_tut4_start


███████████████████████████████████████████████████████████████  
                      CG Membrane builder              
                         Version:  1.2  
███████████████████████████████████████████████████████████████  
---> attempting to obtain point data 
--> inclusion file is provided, we will generate proteins according to this file 
---> point data has been obtained 
---> attempting to generate molecule type 
---> generating molecule types from  input.str  file
--> Note: the lipids will be generated from < Martini Map CG> Lipid Library, of version:  Martini 3
--> This library contains 125 lipid types 
---> molecule types have been generated 
---> reading protein information from the str file 

          |------------------------------------------------------------|
          |     Protein List and ID have been read from the input file |
          |------------------------------------------------------------| 
          |        inclusion with id 1 is mapped to ------> pr

As we can see, we have added another protein of type 2 to our vesicle.

![Image 1](1.png)

But, similarly to tutorial 4, lipids are in the transport protein, which was fixed via exclusions in the earlier tutorial. Let's add exclusions here as well.

In [6]:
#Let's look at the existing exclusions.
point.exclusions.get_all()
#No exclusions found. Let's loop through our proteins of type 2 and add exclusions in the exact same location. We need the point ID as a list:
exclusion_locations=[d["point_id"] for d in point.inclusions.get_by_type(2)]
for id in exclusion_locations:
    point.exclusions.add_pore(point_id=id,radius=1.0)

#See, if it populated the exclusions, and save the folder a long the way for visualization
point.exclusions.get_all()
point.save()

In [7]:
point.exclusions.get_all()

[{'id': 0, 'point_id': 30, 'radius': 1.0},
 {'id': 1, 'point_id': 17, 'radius': 1.0}]

In [8]:
!TS2CG PCG -str input.str -Bondlength 0.2 -LLIB Martini3.LIB -defout system -dts point_tut4_start


███████████████████████████████████████████████████████████████  
                      CG Membrane builder              
                         Version:  1.2  
███████████████████████████████████████████████████████████████  
---> attempting to obtain point data 
--> inclusion file is provided, we will generate proteins according to this file 
--> exclusion file is provided, meaning the system contains pores 
 Note: we are excluding points based on exclusion, If it is slow, contact the developer 
---> point data has been obtained 
---> attempting to generate molecule type 
---> generating molecule types from  input.str  file
--> Note: the lipids will be generated from < Martini Map CG> Lipid Library, of version:  Martini 3
--> This library contains 125 lipid types 
---> molecule types have been generated 
 Note: we are excluding points based on exclusion, If it is slow, contact the developer 
---> reading protein information from the str file 

          |--------------------------

We have successfully removed the lipids from inside the protein by overlaying the exclusions.

![Image 2](2.png)

Now, let's add a secondary domain, somewhere.

We will want to add some POPE lipids in a weighted randomized fashion. While this might not be particularly physical, it will serve the tutorial. We will consider each point and might change the domain to 1, the standard domain is 0. For each point, we will consider the distance to the north pole. The closer to the north pole, the more likely a different domain will be assigned.

In [9]:
# We start by looping over the point and finding the north pole. The north pole will be where the z value is maximum.
id_with_maximum_z=np.argmax(point.outer.coordinates[:,2])
id_with_minimum_z=np.argmin(point.outer.coordinates[:,2])
maximum_distance=np.linalg.norm(point.outer.coordinates[id_with_maximum_z]-point.outer.coordinates[id_with_minimum_z])

for i,Entry in enumerate(point.outer.coordinates):
    distance=np.linalg.norm(Entry-point.outer.coordinates[id_with_maximum_z])
    probability = max(0, .8 - distance / maximum_distance) #The maximum chance of a POPE domain is 80%, decreasing to 0% at the south pole
    if np.random.rand()<probability:
        point.outer.domain_ids[i]=1

point.save()

In order to receive a output from PCG, we need to update the input.str and tell PCG about the new lipid we want in domain 1. This has been done in input_POPE.str So we can run PCG by just refering to the altered str.

In [10]:
!TS2CG PCG -str input_POPE.str -Bondlength 0.2 -LLIB Martini3.LIB -defout system -dts point_tut4_start


███████████████████████████████████████████████████████████████  
                      CG Membrane builder              
                         Version:  1.2  
███████████████████████████████████████████████████████████████  
---> attempting to obtain point data 
--> inclusion file is provided, we will generate proteins according to this file 
--> exclusion file is provided, meaning the system contains pores 
 Note: we are excluding points based on exclusion, If it is slow, contact the developer 
---> point data has been obtained 
---> attempting to generate molecule type 
---> generating molecule types from  input_POPE.str  file
--> Note: the lipids will be generated from < Martini Map CG> Lipid Library, of version:  Martini 3
--> This library contains 125 lipid types 
---> molecule types have been generated 
 Note: we are excluding points based on exclusion, If it is slow, contact the developer 
---> reading protein information from the str file 

          |---------------------

This places POPE lipds accordingly.

![Image 3](3.png)

With this, we have used the point folder manipulater class in TS2CG to change the inclusions, the exclusions, and the domain composition on the fly and tailored to our needs.